## Introduction

### Grap a GPU

Before doing anything, let's try to get a GPU instance.

Go to instance's option:

![step_0](https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/img/step_0_option.png)

Change runtime

![step_1](https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/img/step_1_change_runtime.png)

Add T4 GPU

![step_2](https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/img/step_2_t4.png)

Then connect to a GPU instance

![step_3](https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/img/step_3_connect.png)


### Environment setup

This first cell needs to be executed to set up our environment.

In [ ]:
%pip install ollama
%pip install colab-xterm

!sudo apt-get update
!sudo apt-get install pciutils lshw

!curl -fsSL https://ollama.com/install.sh | sh
url = "https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/client.py"
!wget --no-cache --backups=1 {url}
url = "https://raw.githubusercontent.com/theostos/small-pytanque-tp/refs/heads/main/utils.py"
!wget --no-cache --backups=1 {url}
!pip install requests
!pip install transformers

The following cell instantiate a client to connect to Rocq proof assistant.

In [4]:
from client import ProofAssistantClientAPI, goals_to_str

client = ProofAssistantClientAPI("http://128.93.101.129:8765")

On the server side, there is a file containing some theorems. First, let's look at them.

In [ ]:
for section in client.sections():
  print(f"Theorems in section: {section}")
  for k in range(client.num_thm(section)):
    thm = client.show_thm(section, k)
    print(f"Theorem {k}, {thm['name']} description: {thm['statement']}")
  print()


## A case study: Lelarge's Theorem

We will start by proving the Lelarge's Theorem in the Introduction section.
Let's review it by selecting the first theorem (index = 0) in the "introduction" section.

Before doing so, we need to examine the current context (i.e. the available lemmas, definitions, and notations) to understand it.

In [ ]:
thm =  client.show_thm("introduction", 0)
print("Context:")
print("\n".join(thm['premises']))
print("\n")
print(f"Theorem {thm['name']}: {thm['statement']}",)

Theorem involution_injective states that an arbitrary function f, satisfying Hinv (i.e. being an involution), is one-to-one.

To prove it, we will use the following tactics (read from left to right):

* "intros x y H." introduces two variables, x and y, associated with the forall quantifier, and the hypothesis H corresponding to f x = f y.
* "rewrite <- Hinv with (x := x)." rewrites the left-hand side of the current goal using the hypothesis specialized with x = x.
* "rewrite <- Hinv with (x := y)." rewrites the left-hand side of the current goal using the hypothesis specialized with x = y.
* "rewrite H." rewrites the current goal using the introduced hypothesis H (i.e. f x = f y).
* "reflexivity." discharges goals of the form a = a.

In [ ]:
state, goals = client.start_thm("introduction", 0)
print("Started theorem session:")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "intros x y H.")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "rewrite <- Hinv with (x := x).")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "rewrite <- Hinv with (x := y).")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "rewrite H.")
print(goals_to_str(goals))

In [ ]:
state, goals = client.run_tac(state, "reflexivity.")
print(goals_to_str(goals))

# Automation

## Sledge Hammer

Let's try to automate theorem proving. In deep learning our sledge hammer is big LLM, and one of them is chatGPT.
Is it able to prove Lelarge's theorem?

First, let's ask it to one of the best "reasoning" model: GPT o3-mini.
Extract the sequence of tactics from [this](https://chatgpt.com/share/67e9842e-7d18-8007-b394-d29b03d859cb) link, and try to submit it to the Rocq server ([petanque server](https://github.com/ejgallego/coq-lsp/tree/main/petanque)).

To do it, complete the following cell. You only need to add the remaining tactics

In [ ]:
# complete the following list with the sequence of tactics provided by GPT o3-mini
tactics = ['intros x y H.'] # ['intros x y H.', 'tactic_1', 'tactic_2', 'and so on']

state, goals = client.start_thm("introduction", 0)

for tactic in tactics:
    state, goals = client.run_tac(state, tactic)
    print(goals_to_str(goals))

Remarkably, if we ask more modest model things get a bit more complex:
* Asking to GPT 4o model (see [here](https://chatgpt.com/share/67e986d3-6650-8007-b01c-b5dfe48468a5)), it is able to prove it after 5 attempts.
* Asking to GPT 4o-mini (see [here](https://chatgpt.com/share/67e9871d-9de8-8007-aaeb-04d48d61b525)) is not able to prove it in 8 attempts.

And even more remarkably, both non reasoning and reasoning model from [DeepSeek](https://chat.deepseek.com/) are able to solve this problem at the first attempt.

## Open-weight model

Now, let's try to do it with an open-weight locally run model. We choose Gemma 3 12b, since it has good performance while being able to run on a collab T4 instance.

First we need to start an inference engine (to serve our model locally).

In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])

Then, let's download gemma 3 12b.

In [ ]:
!ollama pull gemma3:12b

To use the model, we will simply call the function get_reponse

In [ ]:
from utils import get_response

prompt = "Why the sky is blue?"

print(get_response(prompt))

Now let's try to prove simple Rocq lemma (section "logic").

In [ ]:
import re

def parse_output(output):
    """
    Function to parse LLM output.
    It expects outputs with the following format
    ```coq
    tactic.
    ```
    """
    # to avoid some parsing issue, we accept instruction to not end with a point as normally required.
    pattern = r'```coq\n(.*?)\.?\n'
    match_output = re.search(pattern, output)
    if match_output:
      output = match_output.group(1).strip()
      return output + '.'
    return ''

prompt_template = """You are an expert in Coq, a theorem-proving assistant. Your task is to help progress a formal proof by providing exactly one correct and effective Coq tactic to advance towards the goal.
Current proof state:
{goal}

Carefully analyze the current goal, consider available hypotheses, and propose the most logical and efficient next step in the proof.

Respond with ONLY ONE Coq tactic enclosed in a Coq code block. Ensure the tactic is syntactically correct and directly applicable. Don't write any comment, simple the code block.

Example of correct formatting:
```coq
tactic_1.
```
"""

for idx in range(client.num_thm('logic')):
  print(f"Try to prove theorem {idx}")
  state, goals = client.start_thm("logic", idx)

  for _ in range(100):
    to_prove_pp = goals_to_str(goals)

    # Gemma3 seems to prefer natural language instead of weird logician symbols
    to_prove = to_prove_pp.replace('|-', 'to prove: ')

    prompt = prompt_template.format(goal=to_prove)
    output = get_response(prompt)
    next_tactic = parse_output(output)

    try:
      state, goals = client.run_tac(state, next_tactic)
      print(next_tactic)
    except Exception as e:
      print("Wrong tactic.")
      pass
    if not goals:
      print("success!")
      break  

Gemma3 seems to make a lot of mistakes.
You may try to improve the prompt.

In the following cell, there is a prompt with an explanation of some tactics in Rocq, try it to see if it makes any difference.


In [ ]:
prompt_template = """You are an expert in Coq, a theorem-proving assistant. Your task is to help progress a formal proof by providing exactly one correct and effective Coq tactic to advance towards the goal.

Here is a brief explanation of tactics you may use:

- intros: Introduces hypotheses or variables into the context. (example: "intros P Q." introduces hypotheses P and Q.)
- apply: Applies a hypothesis or theorem to match the current goal. (example: "apply H0." If the goal is Q and you have a hypothesis H0: P -> Q, applying H0 changes the goal to P.)
- exact: Directly solves the current goal if you have an exact matching hypothesis. (example: "exact H." If the goal is P and you have a hypothesis H: P, then the goal is resolved.)
- contradiction: Resolves the goal if there is a contradiction in the hypotheses. (example: if you have hypotheses H1: P -> False and H2: P, using "contradiction." resolves the goal.)
- unfold not: Expands the definition of negation (~P becomes P -> False). (examples: "unfold not in H." applies it to hypothesis H, "unfold not." applies it to the goal.)
- inversion: Breaks apart hypotheses involving conjunctions (and), disjunctions (or), or existential quantifiers to reveal simpler components. (example: "inversion H." breaks hypothesis H into simpler parts.)
- split: Splits goals involving conjunctions into separate subgoals. (example: "split." transforms goal P /\ Q into two separate goals, P and Q.)
- left/right: Selects a side of a disjunction (or) goal to prove. (examples: "left." to prove the left side of a goal P \/ Q, "right." to prove the right side.)

Current proof state:
{goal}

Carefully analyze the current goal, consider available hypotheses, and propose the most logical and efficient next step in the proof.

Respond with ONLY ONE Coq tactic enclosed in a Coq code block. Ensure the tactic is syntactically correct and directly applicable. Don't write any comment, simple the code block.

Example of correct formatting:
```coq
tactic_1.
```
"""

for idx in range(client.num_thm('logic')):
  print(f"Try to prove theorem {idx}")
  state, goals = client.start_thm("logic", idx)

  for _ in range(100):
    to_prove_pp = goals_to_str(goals)

    # Gemma3 seems to prefer natural language instead of weird logician symbols
    to_prove = to_prove_pp.replace('|-', 'to prove: ')

    prompt = prompt_template.format(goal=to_prove)
    output = get_response(prompt)
    next_tactic = parse_output(output)

    try:
      state, goals = client.run_tac(state, next_tactic)
      print(next_tactic)
    except Exception as e:
      print("Wrong tactic generation.")
      pass
    if not goals:
      print("Success!")
      break  
  print()
  print()

This may not be enough. You could try the following strategies:
- Increase the number of try.
- Add some examples of lemma + proof in your prompt (few-shot prompting).

If you feel confident enough, you could try to improve the overall strategy:
- what would happens if you keep tracks of the wrong step in your prompt? (i.e. the one that throw an error)
- what would happens if you keep tracks of step that leads you to an already visited goal?

In [ ]:
# Find a better strategy/prompt!
prompt_template = """You are an expert in Coq, a theorem-proving assistant. Your task is to help progress a formal proof by providing exactly one correct and effective Coq tactic to advance towards the goal.

Here is a brief explanation of tactics you may use:

- intros: Introduces hypotheses or variables into the context. (example: "intros P Q." introduces hypotheses P and Q.)
- apply: Applies a hypothesis or theorem to match the current goal. (example: "apply H0." If the goal is Q and you have a hypothesis H0: P -> Q, applying H0 changes the goal to P.)
- exact: Directly solves the current goal if you have an exact matching hypothesis. (example: "exact H." If the goal is P and you have a hypothesis H: P, then the goal is resolved.)
- contradiction: Resolves the goal if there is a contradiction in the hypotheses. (example: if you have hypotheses H1: P -> False and H2: P, using "contradiction." resolves the goal.)
- unfold not: Expands the definition of negation (~P becomes P -> False). (examples: "unfold not in H." applies it to hypothesis H, "unfold not." applies it to the goal.)
- inversion: Breaks apart hypotheses involving conjunctions (and), disjunctions (or), or existential quantifiers to reveal simpler components. (example: "inversion H." breaks hypothesis H into simpler parts.)
- split: Splits goals involving conjunctions into separate subgoals. (example: "split." transforms goal P /\ Q into two separate goals, P and Q.)
- left/right: Selects a side of a disjunction (or) goal to prove. (examples: "left." to prove the left side of a goal P \/ Q, "right." to prove the right side.)

Current proof state:
{goal}

Carefully analyze the current goal, consider available hypotheses, and propose the most logical and efficient next step in the proof.

Respond with ONLY ONE Coq tactic enclosed in a Coq code block. Ensure the tactic is syntactically correct and directly applicable. Don't write any comment, simple the code block.

Example of correct formatting:
```coq
tactic_1.
```
"""
for idx in range(client.num_thm('logic')):
  print(f"Try to prove theorem {idx}")
  state, goals = client.start_thm("logic", idx)

  for _ in range(100):
    to_prove_pp = goals_to_str(goals)

    # Gemma3 seems to prefer natural language instead of weird logician symbols
    to_prove = to_prove_pp.replace('|-', 'to prove: ')

    prompt = prompt_template.format(goal=to_prove)
    output = get_response(prompt)
    next_tactic = parse_output(output)

    try:
      state, goals = client.run_tac(state, next_tactic)
      print(next_tactic)
    except Exception as e:
      print("Wrong tactic generation.")
      pass
    if not goals:
      print("Success!")
      break  
  print()
  print()

In the following cell, you will find an implementation that is a bit better (keep track of errors and redundant tactics).

In [ ]:
from collections import defaultdict

prompt_template = """You are an expert in Coq, a theorem-proving assistant. Your task is to help progress a formal proof by providing exactly one correct and effective Coq tactic to advance towards the goal.

Here is a brief explanation of tactics you may use:

- intros: Introduces hypotheses or variables into the context. (example: "intros P Q." introduces hypotheses P and Q.)
- apply: Applies a hypothesis or theorem to match the current goal. (example: "apply H0." If the goal is Q and you have a hypothesis H0: P -> Q, applying H0 changes the goal to P.)
- exact: Directly solves the current goal if you have an exact matching hypothesis. (example: "exact H." If the goal is P and you have a hypothesis H: P, then the goal is resolved.)
- contradiction: Resolves the goal if there is a contradiction in the hypotheses. (example: if you have hypotheses H1: P -> False and H2: P, using "contradiction." resolves the goal.)
- unfold not: Expands the definition of negation (~P becomes P -> False). (examples: "unfold not in H." applies it to hypothesis H, "unfold not." applies it to the goal.)
- inversion: Breaks apart hypotheses involving conjunctions (and), disjunctions (or), or existential quantifiers to reveal simpler components. (example: "inversion H." breaks hypothesis H into simpler parts.)
- split: Splits goals involving conjunctions into separate subgoals. (example: "split." transforms goal P /\ Q into two separate goals, P and Q.)
- left/right: Selects a side of a disjunction (or) goal to prove. (examples: "left." to prove the left side of a goal P \/ Q, "right." to prove the right side.)

Current proof state:
{goal}

Carefully analyze the current goal, consider available hypotheses, and propose the most logical and efficient next step in the proof.

Respond with ONLY ONE Coq tactic enclosed in a Coq code block. Ensure the tactic is syntactically correct and directly applicable. Don't write any comment, simple the code block.

Example of correct formatting:
```coq
tactic_1.
```
"""

prompt_remove = """Don't use any of the following instructions:
{remove}
"""

success = False
for idx in range(client.num_thm('logic')):
  print(f"Try to prove theorem {idx}")
  state, goals = client.start_thm("logic", idx)
  tactics = []
  useless_tactics = defaultdict(list)
  failed_tactics = []
  for _ in range(100):
    to_prove_pp = goals_to_str(goals)
    to_prove = to_prove_pp.replace('|-', 'to prove: ')

    prompt = prompt_template.format(goal=to_prove)
    if failed_tactics:
      prompt += prompt_remove.format(remove="\n".join(failed_tactics))
    
    if useless_tactics[to_prove_pp]:
      prompt += prompt_remove.format(remove="\n".join(useless_tactics[to_prove_pp]))
  
    output = get_response(prompt)
    next_tactic = parse_output(output)
    try:
      state, goals = client.run_tac(state, next_tactic)
      new_to_prove_pp = goals_to_str(goals)
      if to_prove_pp == new_to_prove_pp:
        useless_tactics[new_to_prove_pp].append(next_tactic)
      print(next_tactic)
      tactics.append(next_tactic)
      failed_tactics = []
    except Exception as e:
      failed_tactics.append(next_tactic)
      pass
    if not goals:
      print("Finished!")
      print("Found solution:\n" + "\n".join(tactics))
      break
  if goals:
    print("Failed")
    

## Specialized open-weight model

Let's download [ProofWala](https://arxiv.org/abs/2502.04671), a finetuned version of [Code T5](https://arxiv.org/abs/2109.00859) model on a dataset of Lean and Rocq proofs.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import pipeline
from functools import partial
from utils import generate_tactics_wala

model_name = "amitayusht/ProofWala-Multilingual"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0) # device=0 for GPU, -1 for CPU

generate_tactics = partial(generate_tactics_wala, pipeline, model, tokenizer)

Now, let's try this new model on previous lemmas.

In [ ]:
steps = []
incorrect_steps = []

for idx in range(client.num_thm('logic')):
  print(f"Try to prove theorem {idx}")
  state, goals = client.start_thm("logic", idx)

  for _ in range(100):
    # generate 1 candidate for the next step associated to goals, and given previous steps and incorrect_steps
    tactics, _ = generate_tactics(1, goals, steps=steps, incorrect_steps=incorrect_steps)
    next_tactic = tactics[0]
    try:
      state, goals = client.run_tac(state, next_tactic)
      print(next_tactic)
      steps.append(next_tactic)
      # reset incorrect steps
      incorrect_steps = []
    except Exception as e:
      print("Wrong tactic generation.")
      incorrect_steps.append(next_tactic)
      pass
    if not goals:
      print("success!")
      break  

Now, what would happen with a beam search (step-wise)? Let's try it on our whole collection of exercises.

In [16]:
from utils import beam_search

NUM_TRY = 30
for section in ['introduction', 'logic', 'math']:
    for idx_thm in range(client.num_thm(section)):
        print(f"Trying to prove theorem {idx_thm} in section {section}.")
        for _ in range(NUM_TRY):
            result = beam_search(generate_tactics, client, section, idx_thm, max_depth=7, beam_size=32, timeout=60)
            if result:
                print("Solution found:")
                print("\n".join(result))
                print()
                print()
                break
            else:
                print("FAILED")

ModuleNotFoundError: No module named 'torch'